In [1]:
import sys
sys.path.append("/home/yamato/Application/reduction_tools/")
sys.path.append("/home/yamato/Application/keplerian_mask/")
import reduction_utils_mpi as utils
from keplerian_mask import make_mask
import casatasks
import casatools
from linedictionary_v2 import line_dict
from diskdictionary import disk_dict
from datadictionary import data_dict
import shutil
import os
import analysis_utils as au
import imaging_utils as iutils
import numpy as np

tb = casatools.table()

source = "MWC_480"
line_dict = line_dict[source]
vispath = "./data/measurement_set/"
impath = "./data/image/"
# lines = ["C2H_3-2_fs1", "C2H_3-2_fs2", "C2D_4-3_fs1", "C2D_4-3_fs2"]
lines = ["C2H_1-0_fs1"]
with_cont = True
velocity_range = 10  # maximum velocity w.r.t. vsys in km/s
velocity_width_B3 = 0.5
velocity_width_B67 = 0.2

disk = au.DiskProperty(
    PA=disk_dict[source]["PA_gofish"],
    incl=disk_dict[source]["incl"],
    Mstar=disk_dict[source]["M_star"],
    vsys=disk_dict[source]["v_sys"],
    distance=disk_dict[source]["distance"],
)


You appear to be importing analysisUtils into python (not CASA). version =  3.10.5
CASAPATH is not defined, so I am skipping a lot of imports
Successfully imported `make_mask`.


In [2]:
# split out the respect spectral window and then concatenate
# lines = ["C2H_3-2"]
# lines = ["C2H_3-2_fs2"]
for line in lines:
    data = line_dict[line]["data"]
    if with_cont:
        data += "_wcont"
    ms_list = data_dict[source][data]
    spw = line_dict[line]["spw"]
    if "Band3" in line_dict[line]["data"]:
        velocity_width = velocity_width_B3
    else:
        velocity_width = velocity_width_B67
    print("Base measurement sets: ", ms_list)
    print("Transitions: ", line)
    print("SPW: ", spw)
    ms_list_split = []
    # split
    for ms in ms_list:
        print("Input: ", ms)
        outputvis = vispath + os.path.basename(ms).replace(
            "spectral_line", line
        )
        print("Output: ", outputvis)
        os.system("rm -r " + outputvis)
        casatasks.split(vis=ms, outputvis=outputvis, spw=spw, datacolumn="data")
        ms_list_split.append(outputvis)

    # concat
    concatvis = vispath + f"{source}_{line}.ms"
    if with_cont:
        concatvis = concatvis.replace(".ms", "_wcont.ms")
    print("Concatenated MS: ", concatvis)
    os.system("rm -r " + concatvis)
    casatasks.concat(vis=ms_list_split, concatvis=concatvis)
    for vis in ms_list_split:
        shutil.rmtree(vis)

    # cvel
    print("cveled MS: ", concatvis + ".cvel")
    print("Channel width: ", velocity_width, " km/s")
    print(
        "Velocity range: +/-",
        velocity_range,
        " km/s w.r.t. v_sys = ",
        disk.vsys,
        " km/s",
    )
    # calculate velocity range
    nu0 = float(line_dict[line]["restfreq"].replace("GHz", "")) * 1e9
    freqs = np.array(
        [float(nu.replace("GHz", "")) * 1e9 for nu in line_dict[line]["freqs"]]
    )
    v0 = au.dnu2dv(dnu=freqs - nu0, nu0=nu0) + disk.vsys
    vmin = np.min(v0) - velocity_range
    vmax = np.max(v0) + velocity_range
    vmin = np.copysign(np.ceil(np.abs(vmin)), vmin)
    vmax = np.copysign(np.ceil(np.abs(vmax)), vmax)
    os.system("rm -r " + concatvis + ".cvel")
    casatasks.cvel2(
        vis=concatvis,
        outputvis=concatvis + ".cvel",
        mode="velocity",
        restfreq=line_dict[line]["restfreq"],
        outframe="LSRK",
        veltype="radio",
        width=f"{velocity_width}km/s",
        start=f"{float(vmin)}km/s",
        nchan=int((vmax - vmin) / velocity_width),
    )

    # set the rest frequency
    tb.open(concatvis + ".cvel/SOURCE", nomodify=False)
    tb.putcol("REST_FREQUENCY", np.array([[nu0]]))
    tb.close()

Base measurement sets:  ['/works/yamato/MAPS_data/MWC_480/Band3/self-calibrated/MWC_480_LB1_spectral_line_wcont.ms', '/works/yamato/MAPS_data/MWC_480/Band3/self-calibrated/MWC_480_LB2_spectral_line_wcont.ms', '/works/yamato/MAPS_data/MWC_480/Band3/self-calibrated/MWC_480_LB3_spectral_line_wcont.ms', '/works/yamato/MAPS_data/MWC_480/Band3/self-calibrated/MWC_480_LB4_spectral_line_wcont.ms', '/works/yamato/MAPS_data/MWC_480/Band3/self-calibrated/MWC_480_LB6_spectral_line_wcont.ms']
Transitions:  C2H_1-0_fs1
SPW:  0
Input:  /works/yamato/MAPS_data/MWC_480/Band3/self-calibrated/MWC_480_LB1_spectral_line_wcont.ms
Output:  ./data/measurement_set/MWC_480_LB1_C2H_1-0_fs1_wcont.ms


rm: './data/measurement_set/MWC_480_LB1_C2H_1-0_fs1_wcont.ms' を削除できません: そのようなファイルやディレクトリはありません


Input:  /works/yamato/MAPS_data/MWC_480/Band3/self-calibrated/MWC_480_LB2_spectral_line_wcont.ms
Output:  ./data/measurement_set/MWC_480_LB2_C2H_1-0_fs1_wcont.ms


rm: './data/measurement_set/MWC_480_LB2_C2H_1-0_fs1_wcont.ms' を削除できません: そのようなファイルやディレクトリはありません


Input:  /works/yamato/MAPS_data/MWC_480/Band3/self-calibrated/MWC_480_LB3_spectral_line_wcont.ms
Output:  ./data/measurement_set/MWC_480_LB3_C2H_1-0_fs1_wcont.ms


...

rm: './data/measurement_set/MWC_480_LB6_C2H_1-0_fs1_wcont.ms' を削除できません: そのようなファイルやディレクトリはありません


Concatenated MS:  ./data/measurement_set/MWC_480_C2H_1-0_fs1_wcont.ms


rm: './data/measurement_set/MWC_480_C2H_1-0_fs1_wcont.ms' を削除できません: そのようなファイルやディレクトリはありません


cveled MS:  ./data/measurement_set/MWC_480_C2H_1-0_fs1_wcont.ms.cvel
Channel width:  0.5  km/s
Velocity range: +/- 10  km/s w.r.t. v_sys =  5.1  km/s


rm: './data/measurement_set/MWC_480_C2H_1-0_fs1_wcont.ms.cvel' を削除できません: そのようなファイルやディレクトリはありません


In [3]:
# first make dirty image and then CLEAN
### basic parameters for tclean ###
cellsize = "0.02arcsec"
weighting = "superuniform"
scales = [0, 10, 30]
npixels = 0
target_bmaj = 0.3
deconvolver = "multiscale"
spw = "0"
perchanweightdensity = True
max_imsize = 1920

### parameters for Keplerian mask ###
r_min = 0.25
r_max = 2.5
zr = 0.0
nbeams = 1.5

# lines = ["DCN_4-3"]
for line in lines:
    print("Processing " + line)
    # first calculate the taper parameter to acheive the desired beams
    vis = vispath + f"{source}_{line}.ms.cvel"
    imagename = impath + f"{source}_{line}_hybrid"
    if with_cont:
        vis = vis.replace(".ms", "_wcont.ms")
        imagename += "_wcont"
    imsize = utils.get_imsize(
        vis=vis,
        field=source,
        cellsize=cellsize,
    )
    if imsize > max_imsize:
        imsize = max_imsize

    for ext in [".alpha", ".image", ".mask", ".model", ".pb", ".psf", ".residual", ".sumwt"]:
        if os.path.exists(imagename + ext):
            shutil.rmtree(imagename + ext)
            
    uvtaper = iutils.calc_uvtaper(
        vis=vis,
        imagename=imagename,
        field=source,
        specmode="cube",
        weighting=weighting,
        perchanweightdensity=perchanweightdensity,
        npixels=npixels,
        cellsize=cellsize,
        imsize=imsize,
        target_beam=(target_bmaj, target_bmaj * np.cos(np.deg2rad(disk.incl)), disk.PA),
    )

    for ext in [".alpha", ".image", ".mask", ".model", ".pb", ".psf", ".residual", ".sumwt"]:
        if os.path.exists(imagename + ext):
            shutil.rmtree(imagename + ext)

    casatasks.tclean(
        vis=vis,
        imagename=imagename,
        spw=spw,
        specmode="cube",
        outframe="LSRK",
        veltype="radio",
        imsize=imsize,
        cell=cellsize,
        scales=scales,
        weighting=weighting,
        perchanweightdensity=perchanweightdensity,
        npixels=npixels,
        uvtaper=uvtaper,
        restoringbeam="common",
        niter=0
    )

    # make mask
    rms = make_mask(
        imagename=imagename + ".image",
        inc=disk.incl,
        PA=disk.PA,
        mstar=disk.Mstar,
        dist=disk.distance,
        vlsr=disk.vsys * 1e3,
        nbeams=nbeams,
        r_min=r_min,
        r_max=r_max,
        zr=zr,
        restfreqs=line_dict[line]["freqs"],
        export_FITS=True,
        overwrite=True,
    )
    # shutil.rmtree(imagename + ".mask.image")

    # CLEAN
    # rms = utils.calc_sensitivity(
    #     vis,
    #     cellsize=cellsize,
    #     imsize=imsize,
    #     weighting=weighting,
    #     npixels=npixels,
    #     specmode="cube",
    #     spw=[spw],
    #     chan=int(0.5 * (2 * velocity_range / velocity_width)),
    # )

    for ext in [".alpha", ".image", ".mask", ".model", ".pb", ".psf", ".residual", ".sumwt"]:
        if os.path.exists(imagename + ext):
            shutil.rmtree(imagename + ext)

    casatasks.tclean(
        vis=vis,
        imagename=imagename,
        spw=spw,
        specmode="cube",
        outframe="LSRK",
        veltype="radio",
        imsize=imsize,
        cell=cellsize,
        scales=scales,
        weighting=weighting,
        perchanweightdensity=perchanweightdensity,
        npixels=npixels,
        uvtaper=uvtaper,
        restoringbeam="common",
        niter=1000000,
        threshold=f"{3*rms}Jy",
        usemask="user",
        mask=imagename + ".mask.image"
    )

    # pbcor
    print("Correcting primary beam response")
    casatasks.impbcor(
        imagename=imagename + ".image",
        pbimage=imagename + ".pb",
        outfile=imagename + ".image.pbcor",
        overwrite=True,
    )

    # export fits
    print("Exporting to FITS")
    for ext in [".image", ".image.pbcor"]:
        casatasks.exportfits(
            imagename=imagename + ext,
            fitsimage=imagename + ext + ".fits",
            dropstokes=True,
            overwrite=True,
        )

Processing C2H_1-0_fs1
Starting to estimate the uvtaper which achieve the target beam of 0.3000 arcsec x 0.2396 arcsec (P.A. = 328.0000 deg)
Pixel size: 0.02arcsec
Image size: 1920
Calculating and fitting PSF with the original weighting scheme (weighting = superuniform)



0%....10....20....30....40....50....60....70....80....90....100%


Done.
(1920, 1920)
Restoring beam shape: 0.1510 arcsec x 0.1013 arcsec (P.A. = -24.7057 deg)
Windowing out the central region for computational efficiency
Image size of windowed psf: 600
Calculating uvtaper parameter
Done. Best-fit uvtaper parameter: ['0.27872339340721985arcsec', '0.21465667336199573arcsec', '317.4644103934708deg']
Calculating the resulting beam shape after uvtaper



0%....10....20....30....40....50....60....70....80....90....100%


Done.
Restoring beam shape after uvtaper: 0.3000 arcsec x 0.2397 arcsec (P.A. = -31.7527 deg)
JvM epsilon: 0.9505



0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%
2024-01-27 12:57:48	WARN	task_tclean::SIImageStore::restore (file /source/casa6/casatools/src/code/synthesis/ImagerObjects/SIImageStore.cc, line 2284)	Restoring with an empty model image. Only residuals will be processed to form the output restored image.
....10....20....30....40....50....60....70....80....90....100%
....10....20....30....40....50....60....70....80....90....100%


# Estimated RMS of unmasked regions: 1.37 mJy/beam
# If there are strong sidelobes this may overestimate the RMS.



0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%


Correcting primary beam response
Exporting to FITS
